# Data Cleaning Basics 

This is an exercise to familiarize myself to selecting, assigning, and analyzing data with pandas using pre-cleaned dataset. I'm going to work with the `laptops.csv` dataset, which contains information about 1300 laptop computers.

In [1]:
import pandas as pd

laptops = pd.read_csv('laptops.csv', encoding = 'Latin-1')    
# there was an error reading the file using UTF-8 encoding, therefore, Latin-1 will be used
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Manufacturer              1303 non-null   object
 1   Model Name                1303 non-null   object
 2   Category                  1303 non-null   object
 3   Screen Size               1303 non-null   object
 4   Screen                    1303 non-null   object
 5   CPU                       1303 non-null   object
 6   RAM                       1303 non-null   object
 7    Storage                  1303 non-null   object
 8   GPU                       1303 non-null   object
 9   Operating System          1303 non-null   object
 10  Operating System Version  1133 non-null   object
 11  Weight                    1303 non-null   object
 12  Price (Euros)             1303 non-null   object
dtypes: object(13)
memory usage: 132.5+ KB


### Cleaning column names

We can see that every column is represented as the `object` type, indicating that they are represented by strings, not numbers. Also, one of the columns, `Operating System Version`, has null values.

One noticeable issue is that the `" Storage"` column name has a space in front of it. These quirks with column labels can sometimes be hard to spot, so removing extra whitespaces from all column names will save us more work in the long run.

In [2]:
new_columns = []
for col_name in laptops.columns:
    col_name = col_name.strip()
    new_columns.append(col_name)
    
laptops.columns = new_columns

The columns still consist of variety of upper and lowercase letters, as well as parentheses, which will make them harder to work with or read. We can create a function that uses Python string methods to clean our column labels, and then again use a loop to apply that function to each label.

In [3]:
def clean_col(col_name):
    col_name = col_name.replace("Operating System", "os") 
    col_name = col_name.strip()
    col_name = col_name.replace(')','')
    col_name = col_name.replace('(','')
    col_name = col_name.lower()
    col_name = col_name.replace(' ','_')
    return col_name

new_columns = []
for col_name in laptops.columns:
    new_columns.append(clean_col(col_name))

laptops.columns = new_columns
print(laptops.columns)

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')


### Converting string columns to numeric

Of these three columns, we have three different types of text data:

- `category`: Purely text data - there are no numeric values.
- `screen_size`: Numeric data stored as text data because of the `"` character.
- `screen`: A combination of pure text data with numeric data.

In [4]:
print(laptops.iloc[:5,2:5])

    category screen_size                              screen
0  Ultrabook       13.3"  IPS Panel Retina Display 2560x1600
1  Ultrabook       13.3"                            1440x900
2   Notebook       15.6"                   Full HD 1920x1080
3  Ultrabook       15.4"  IPS Panel Retina Display 2880x1800
4  Ultrabook       13.3"  IPS Panel Retina Display 2560x1600


In [5]:
laptops["screen_size"] = laptops["screen_size"].str.replace('"','')
laptops["screen_size"] = laptops["screen_size"].astype(float)
print(laptops["screen_size"].dtype)
print(laptops["screen_size"].unique())

# Changing 'screen_size' column name:
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

float64
[13.3 15.6 15.4 14.  12.  11.6 17.3 10.1 13.5 12.5 13.  18.4 13.9 12.3
 17.  15.  14.1 11.3]


In [6]:
unique_ram = laptops['ram'].unique()
print(unique_ram)

['8GB' '16GB' '4GB' '2GB' '12GB' '6GB' '32GB' '24GB' '64GB']


In [7]:
laptops['ram'] = laptops['ram'].str.replace('GB$', '', regex = True).astype(int)

unique_ram = laptops['ram'].unique()
print(unique_ram)

[ 8 16  4  2 12  6 32 24 64]


In [8]:
laptops.rename({'ram':'ram_gb'}, axis = 1 , inplace = True)

In [9]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   manufacturer        1303 non-null   object 
 1   model_name          1303 non-null   object 
 2   category            1303 non-null   object 
 3   screen_size_inches  1303 non-null   float64
 4   screen              1303 non-null   object 
 5   cpu                 1303 non-null   object 
 6   ram_gb              1303 non-null   int32  
 7   storage             1303 non-null   object 
 8   gpu                 1303 non-null   object 
 9   os                  1303 non-null   object 
 10  os_version          1133 non-null   object 
 11  weight              1303 non-null   object 
 12  price_euros         1303 non-null   object 
dtypes: float64(1), int32(1), object(11)
memory usage: 127.4+ KB


### Extracting values from string

Sometimes, it can be useful to extract non-numeric values from within strings. Let's look at the first five values from the `gpu` (graphics processing unit) column:

In [10]:
print(laptops["gpu"].head())

0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
3              AMD Radeon Pro 455
4    Intel Iris Plus Graphics 650
Name: gpu, dtype: object


In [11]:
laptops["gpu_manufacturer"] = (laptops["gpu"].str.split().str[0])

laptops['cpu_manufacturer'] = (laptops['cpu'].str.split().str[0])

cpu_manufacturer_counts = laptops['cpu_manufacturer'].value_counts()

### Correcting bad values

In [12]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

laptops['os'] = laptops['os'].map(mapping_dict)
laptops['os'].value_counts()

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          21
Android         2
Name: os, dtype: int64

### Dropping missing values

In [13]:
print(laptops.isnull().sum())

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight                  0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64


It's now clear that we have only one column with null values, `os_version`, which has 170 missing values.

In [14]:
value_counts_before = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
print(value_counts_before)

laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"

laptops.loc[laptops['os'] == "No OS", "os_version"] = "Version Unknown"

value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64


In [15]:
print(value_counts_after)

Linux        62
Chrome OS    27
Android       2
Name: os, dtype: int64


### Clean a string column

In [16]:
laptops['weight'].unique()

array(['1.37kg', '1.34kg', '1.86kg', '1.83kg', '2.1kg', '2.04kg', '1.3kg',
       '1.6kg', '2.2kg', '0.92kg', '1.22kg', '0.98kg', '2.5kg', '1.62kg',
       '1.91kg', '2.3kg', '1.35kg', '1.88kg', '1.89kg', '1.65kg',
       '2.71kg', '1.2kg', '1.44kg', '2.8kg', '2kg', '2.65kg', '2.77kg',
       '3.2kg', '0.69kg', '1.49kg', '2.4kg', '2.13kg', '2.43kg', '1.7kg',
       '1.4kg', '1.8kg', '1.9kg', '3kg', '1.252kg', '2.7kg', '2.02kg',
       '1.63kg', '1.96kg', '1.21kg', '2.45kg', '1.25kg', '1.5kg',
       '2.62kg', '1.38kg', '1.58kg', '1.85kg', '1.23kg', '1.26kg',
       '2.16kg', '2.36kg', '2.05kg', '1.32kg', '1.75kg', '0.97kg',
       '2.9kg', '2.56kg', '1.48kg', '1.74kg', '1.1kg', '1.56kg', '2.03kg',
       '1.05kg', '4.4kg', '1.90kg', '1.29kg', '2.0kg', '1.95kg', '2.06kg',
       '1.12kg', '1.42kg', '3.49kg', '3.35kg', '2.23kg', '4.42kg',
       '2.69kg', '2.37kg', '4.7kg', '3.6kg', '2.08kg', '4.3kg', '1.68kg',
       '1.41kg', '4.14kg', '2.18kg', '2.24kg', '2.67kg', '2.14kg',
       '1.

In [17]:
laptops['weight'] = laptops['weight'].str.replace("[a-z]*", "", regex = True)

laptops['weight'] = laptops['weight'].astype(float)

laptops.rename({"weight":"weight_kg"}, axis =1, inplace =True)

laptops.to_csv("laptops_cleaned.csv", index = False)

Our dataset is ready for some analysis, but there are still some data cleaning tasks left! Here are your next steps:

- Convert the price_euros column to a numeric dtype.
- Extract the screen resolution from the screen column.
- Extract the processor speed from the cpu column.


Here are some questions you might like to answer in your own time by analyzing the cleaned data:

- Are laptops made by Apple more expensive than those made by other manufacturers?
- What is the best value laptop with a screen size of 15" or more?
- Which laptop has the most storage space?

#### Convert `price_euros` column to numeric dtype

In [18]:
laptops['price_euros'].head()

0    1339,69
1     898,94
2     575,00
3    2537,45
4    1803,60
Name: price_euros, dtype: object

Because they are treated as `object` in a `Series`, we are going to replace the `,` with `.`,and convert the column to float dtype.

In [19]:
laptops['price_euros'] = laptops['price_euros'].str.replace(',','.')
laptops['price_euros'] = laptops['price_euros'].astype(float)

In [20]:
laptops['price_euros'].head()

0    1339.69
1     898.94
2     575.00
3    2537.45
4    1803.60
Name: price_euros, dtype: float64

#### Extract screen resolution from the screen column

In [21]:
laptops[['screen_size_inches', 'screen']].head()

,screen_size_inches,screen
0,13.3,IPS Panel Retina Display 2560x1600
1,13.3,1440x900
2,15.6,Full HD 1920x1080
3,15.4,IPS Panel Retina Display 2880x1800
4,13.3,IPS Panel Retina Display 2560x1600


Looking at the `screen` column, the `screen solution` is some numbers, plus the `x`, plus some numbers. We can extract this by using `split` function. Then we can add one more column to the dataset reporting only screen resolution.

In [22]:
screen_reso = laptops['screen'].str.split().str[-1]
print(screen_reso)

0       2560x1600
1        1440x900
2       1920x1080
3       2880x1800
4       2560x1600
          ...    
1298    1920x1080
1299    3200x1800
1300     1366x768
1301     1366x768
1302     1366x768
Name: screen, Length: 1303, dtype: object


In [23]:
laptops['screen_reso'] = screen_reso
laptops[['screen_reso', 'screen']].head()

,screen_reso,screen
0,2560x1600,IPS Panel Retina Display 2560x1600
1,1440x900,1440x900
2,1920x1080,Full HD 1920x1080
3,2880x1800,IPS Panel Retina Display 2880x1800
4,2560x1600,IPS Panel Retina Display 2560x1600


#### Extract processor speed from cpu column

In [24]:
laptops['cpu'].head()

0          Intel Core i5 2.3GHz
1          Intel Core i5 1.8GHz
2    Intel Core i5 7200U 2.5GHz
3          Intel Core i7 2.7GHz
4          Intel Core i5 3.1GHz
Name: cpu, dtype: object

The speed of the `cpu` is a float, followed by `GHz` without spacing. We can use `split` function to make this work, then add to a new column

In [25]:
cpu_speed = laptops['cpu'].str.split().str[-1]
laptops['cpu_speed'] = cpu_speed.str.replace('GHz','').astype(float)
laptops['cpu_speed'].head()

0    2.3
1    1.8
2    2.5
3    2.7
4    3.1
Name: cpu_speed, dtype: float64

In [26]:
laptops.to_csv("laptops_cleaned.csv", index = False)

### Answer questions using cleaned dataset

#### Are laptops made by Apple more expensive than those made by other manufacturers?

In [27]:
laptops.head()

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,gpu_manufacturer,cpu_manufacturer,screen_reso,cpu_speed
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,X,1.37,1339.69,Intel,Intel,2560x1600,2.3
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,X,1.34,898.94,Intel,Intel,1440x900,1.8
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,Version Unknown,1.86,575.00,Intel,Intel,1920x1080,2.5
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,X,1.83,2537.45,AMD,Intel,2880x1800,2.7
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,X,1.37,1803.60,Intel,Intel,2560x1600,3.1


In [28]:
apple = laptops[laptops['manufacturer'] == 'Apple']
non_apple = laptops[~(laptops['manufacturer'] == 'Apple')]

In [29]:
apple_vs_nonapple = pd.DataFrame( {'Mean':[apple['price_euros'].mean(), non_apple['price_euros'].mean()] 
                                  ,'Median':[apple['price_euros'].median(), non_apple['price_euros'].median()]}
                                 ,index = ['Apple', 'Non Apple'])

In [30]:
apple_vs_nonapple

,Mean,Median
Apple,1564.198571,1339.69
Non Apple,1116.471123,959.50


As we can see from the `apple_vs_nonapple` dataframe, `Apple` products' prices are above that of `Non Apple` products, both in terms of `mean` and `median`. 

#### What is the best value laptop with a screen size of 15" or more?

In [31]:
screen_more_15 = laptops[laptops['screen_size_inches'] >= 15]

In [33]:
lst = ['manufacturer', 'model_name', 'screen_size_inches', 'ram_gb', 'price_euros', 'cpu_speed']
best_price = screen_more_15.loc[screen_more_15['price_euros'] == screen_more_15['price_euros'].min(), lst]
best_cpu_speed = screen_more_15.loc[screen_more_15['cpu_speed'] == screen_more_15['cpu_speed'].max(), lst]
best_ram_gb = screen_more_15.loc[screen_more_15['ram_gb'] == screen_more_15['ram_gb'].max(), lst]
big_screen = screen_more_15.loc[screen_more_15['screen_size_inches'] == screen_more_15['screen_size_inches'].max(), lst]

# Creating a DataFrame
#data = [['best_price', best_price], ['best_cpu_speed',best_cpu_speed], ['best_ram_gb', best_ram_gb], ['big_screen',big_screen]]
#best_value_laptop = pd.DataFrame(data, index = ['best_price', 'best_cpu_speed', 'best_ram_gb', 'big_screen'], columns = lst)

manufacturer = pd.Series(best_price[0], best_cpu_speed[0], best_ram_gb[0], big_screen[0])
model_name = pd.Series(best_price[1], best_cpu_speed[1], best_ram_gb[1], big_screen[1])
screen_size = pd.Series(best_price[2], best_cpu_speed[2], best_ram_gb[2], big_screen[2])
ram_gb = pd.Series(best_price[3], best_cpu_speed[3], best_ram_gb[3], big_screen[3])
price_euros = pd.Series(best_price[4], best_cpu_speed[4], best_ram_gb[4], big_screen[4])
cpu_speed = pd.Series(best_price[5], best_cpu_speed[5], best_ram_gb[5], big_screen[5])

best_value_laptop = pd.concat([manufacturer, model_name, screen_size, ram_gb, price_euros, cpu_speed],axis = 1 )

KeyError: 0

In [ ]:
best_value_laptop